In [ ]:
import pandas as pd

# df = pd.read_csv("hf://datasets/jettisonthenet/timeseries_trending_youtube_videos_2019-04-15_to_2020-04-15/videostats.csv")
df = pd.read_csv("youtube_dataset.csv")
df.head()

,videostatsid,ytvideoid,views,comments,likes,dislikes,timestamp
0,1,0GLbwkfhYZk,5722228,28229,185188,10860,2019-04-15 04:42:49
1,2,O4NB73HTlxI,3673770,31565,278477,4705,2019-04-15 04:42:50
2,3,VVRcXtSjPcE,253613,1025,4587,87,2019-04-15 04:42:51
3,4,adzYW5DZoWs,21567972,79208,478496,61703,2019-04-15 04:42:51
4,5,p_8yK2kmxoo,2385064,10703,133304,1323,2019-04-15 04:42:52


# Explore data

In [93]:
df[df.duplicated()]

,videostatsid,ytvideoid,views,comments,likes,dislikes,timestamp


In [94]:
df[df.duplicated(subset=['videostatsid'])]

,videostatsid,ytvideoid,views,comments,likes,dislikes,timestamp


In [95]:
df.loc[df.isna().any(axis=1)]


,videostatsid,ytvideoid,views,comments,likes,dislikes,timestamp


In [96]:
len(df['ytvideoid'].unique())

11369

In [97]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['timestamp'] = df['timestamp'].dt.round('30min')

df = df.sort_values(['ytvideoid', 'timestamp'])

In [98]:
df.groupby('ytvideoid')['timestamp'].count().describe()

count    11369.000000
mean       135.555282
std         77.114336
min          1.000000
25%         84.000000
50%        136.000000
75%        181.000000
max        628.000000
Name: timestamp, dtype: float64

In [99]:
df['time_diff'] = df.groupby('ytvideoid')['timestamp'].diff()
df['time_diff'].describe()

count                      1529759
mean     0 days 00:29:53.520155789
std      0 days 00:44:15.257426654
min                0 days 00:00:00
25%                0 days 00:30:00
50%                0 days 00:30:00
75%                0 days 00:30:00
max                7 days 19:00:00
Name: time_diff, dtype: object

# Clean data

In [100]:
df_30min = df[df['time_diff'] == pd.Timedelta('30min')]
first_rows = df.groupby('ytvideoid').head(1).index
df = df.loc[df_30min.index.union(first_rows)]
df.drop(columns='time_diff', inplace=True)
df.head()

,videostatsid,ytvideoid,views,comments,likes,dislikes,timestamp
0,1,0GLbwkfhYZk,5722228,28229,185188,10860,2019-04-15 04:30:00
1,2,O4NB73HTlxI,3673770,31565,278477,4705,2019-04-15 04:30:00
2,3,VVRcXtSjPcE,253613,1025,4587,87,2019-04-15 04:30:00
3,4,adzYW5DZoWs,21567972,79208,478496,61703,2019-04-15 04:30:00
4,5,p_8yK2kmxoo,2385064,10703,133304,1323,2019-04-15 04:30:00


In [101]:
min_rows = 48
df = df.groupby('ytvideoid').filter(lambda x: len(x) >= min_rows)
df = df.sort_values(['ytvideoid', 'timestamp'])
df

,videostatsid,ytvideoid,views,comments,likes,dislikes,timestamp
2802,2803,--Oly6yY5To,354013,2472,8037,1208,2019-04-15 11:30:00
2901,2902,--Oly6yY5To,355038,2486,8083,1220,2019-04-15 12:00:00
3099,3100,--Oly6yY5To,358091,2494,8154,1242,2019-04-15 12:30:00
3298,3299,--Oly6yY5To,361332,2516,8231,1265,2019-04-15 13:00:00
3496,3497,--Oly6yY5To,364443,2547,8304,1288,2019-04-15 13:30:00
...,...,...,...,...,...,...,...
604270,604271,zzrAx_he5oY,753130,5237,41054,717,2019-08-20 08:30:00
604370,604371,zzrAx_he5oY,753544,5237,41064,718,2019-08-20 09:00:00
604470,604471,zzrAx_he5oY,753974,5240,41079,717,2019-08-20 09:30:00
604570,604571,zzrAx_he5oY,754419,5240,41110,717,2019-08-20 10:00:00


In [102]:
df.drop(columns=['videostatsid'], inplace=True)
df['ytvideoid'] = df['ytvideoid'].astype('category').cat.codes
df

,ytvideoid,views,comments,likes,dislikes,timestamp
2802,0,354013,2472,8037,1208,2019-04-15 11:30:00
2901,0,355038,2486,8083,1220,2019-04-15 12:00:00
3099,0,358091,2494,8154,1242,2019-04-15 12:30:00
3298,0,361332,2516,8231,1265,2019-04-15 13:00:00
3496,0,364443,2547,8304,1288,2019-04-15 13:30:00
...,...,...,...,...,...,...
604270,9745,753130,5237,41054,717,2019-08-20 08:30:00
604370,9745,753544,5237,41064,718,2019-08-20 09:00:00
604470,9745,753974,5240,41079,717,2019-08-20 09:30:00
604570,9745,754419,5240,41110,717,2019-08-20 10:00:00


# Split data

In [118]:
from sklearn.model_selection import train_test_split

train_ratio = 0.75
val_ratio = 0.15
test_ratio = 0.1

videos_id = df['ytvideoid'].unique()

train_ids, test_ids = train_test_split(videos_id, test_size = 1-train_ratio)
val_ids, test_ids = train_test_split(test_ids, test_size = test_ratio/(test_ratio+val_ratio))

train_df = df[df['ytvideoid'].isin(train_ids)]
val_df = df[df['ytvideoid'].isin(val_ids)]
test_df = df[df['ytvideoid'].isin(test_ids)]

print(f'Train size: {len(train_df)}\nVal size: {len(val_df)}\nTest size: {len(test_df)}')

Train size: 1069449
Val size: 213756
Test size: 142088


In [ ]:
train_df.to_csv('train_data.csv', index=False)
val_df.to_csv('val_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)